In [73]:
import random
import math
import numpy as np

In [74]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]


**Definition of the class Individual**

In [99]:
class Individual:
    def __init__(self, genome: list, space: np.array):
        self.genome = np.array(genome)
        list_covered_elm = space[np.where(self.genome == 1)[0]]
        cost = sum(list(map(lambda x: len(x), list_covered_elm)))
        covered_elm = len(np.unique(np.hstack(list_covered_elm)))
        self.fitness = (covered_elm, PROBLEM_SIZE-cost)

    def weight(self):
      return -(self.fitness[1]-PROBLEM_SIZE)
    
    def get_covered_elements(self):
      list_covered_elm = space[np.where(self.genome == 1)[0]]
      return list_covered_elm

**Hyperparameters**

In [105]:
PROBLEM_SIZE = 500 #define the problem size
POPULATION_SIZE = 2*PROBLEM_SIZE #tuned
NUM_GENERATIONS = 800 #tuned
MUTATION_RATE = 0.35 #tuned
OFFSPRING_SIZE = math.ceil(1.5*PROBLEM_SIZE)
space = problem(PROBLEM_SIZE, seed=42)
space = np.array([np.asarray(xi) for xi in space], dtype=set)
GENOME_SIZE = len(space)

**Generate Population of Individuals**

In [106]:
population = [Individual(genome=[random.choice([0, 1]) for _ in range(GENOME_SIZE)], space=space) for i in range(POPULATION_SIZE)]

**Define GA operators**

In [107]:
def tournament(population, tournament_size=2):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)


def crossover(g1, g2):
    cut = random.randint(0, len(g1))
    return list(np.concatenate((g1[:cut],g2[cut:])))
    #return g1[:cut] + g2[cut:]


def mutation(g):
    point = random.randint(0, len(g) - 1)
    return list(np.concatenate((g[:point],np.array((1 - g[point],)), g[point + 1 :])))
    #return g[:point] + list((1 - g[point],)) + g[point + 1 :]

**GA algorithm**

In [ ]:
from re import MULTILINE
for generation in range(NUM_GENERATIONS):
    offspring = list()
    genomes = list(map(lambda x: list(x.genome), population))
    for i in range(OFFSPRING_SIZE):
        p1 = tournament(population)
        p2 = tournament(population)
        genome = crossover(p1.genome, p2.genome)
        if random.random() < MUTATION_RATE:
             genome = mutation(genome)
        if genome not in genomes:
          individual = Individual(genome=genome, space=space)
          offspring.append(individual)
    population += offspring
    population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]

**Report Metrics**

In [ ]:
print(
    f"-N: {PROBLEM_SIZE}: "
    f"Total weight: {population[0].weight()}; "
    f"(bloat={(population[0].weight()-PROBLEM_SIZE)/PROBLEM_SIZE*100:.0f}%)"
)